# Asking LLMs to tell a joke and conversation between models
It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models. Later we will be putting LLMs to better use!

In [2]:
# Importing the Libraries

import requests
import json
import time
import re
from typing import List
from IPython.display import Markdown, display, update_display

There are other parameters that can be used, including temperature which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [3]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

fun_prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [4]:
OLLAMA_URL = "http://localhost:11434/api/chat"

In [5]:
def model_test(model_name:str, prompts):
    payload = {
        "model": model_name,
        "messages": prompts,
        "temperature": 0.8,
        "stream": False  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    reply = response.json()["message"]["content"]
    reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    print(f"Model Name: {model_name}\n")
    display(Markdown(reply))
    print("Total Time Taken: ", time.time()-start_time, "\n\n")

In [6]:
model_list = ["mistral", "llama3.1:latest"] # get this from ollama list

In [7]:
for model in model_list:
    model_test(model, fun_prompts)

Model Name: mistral



Here's a humorous data scientist joke for you:

Why was the pies-and-coffee shop such a great place for data scientists?

Because it had a mean and median for every pie, and the mode was always free on Tuesdays!

Data science humor at its finest! 😉

Total Time Taken:  18.667943239212036 


Model Name: llama3.1:latest



Why did the neural network go on a diet?

Because it wanted to lose some layers! (get it?)

Hope that one trained a smile on your face!

Total Time Taken:  14.489516258239746 




In [8]:
business_prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [9]:
for model in model_list:
    model_test(model, business_prompts)

Model Name: mistral



To determine if a business problem is suitable for a solution using Language Model Learning (LLM), consider the following factors:

1. **Natural Language Understanding**: If your problem involves analyzing, understanding, and generating human language, such as text classification, sentiment analysis, or chatbot development, then it may be a good fit for LLM.

2. **Structured Data Processing**: If the business problem requires processing and analyzing structured data like tables, graphs, or numerical data, other machine learning models or traditional statistical methods might be more suitable. However, if the data is unstructured text or speech that needs to be converted into a usable format before analysis, LLM could still potentially help with preprocessing steps.

3. **Data Availability**: Adequate training data is essential for any machine learning model, including LLM. If you have a large dataset of relevant text or speech to train the model on, it may be more suitable for an LLM approach.

4. **Complexity and Scalability**: If your business problem involves complex linguistic patterns, context, or nuances, an LLM solution might be appropriate, especially as many state-of-the-art models can handle a wide range of language tasks at scale. However, it's essential to consider the computational resources required for training and deployment.

5. **Integration with Existing Systems**: If your business problem involves integrating AI solutions into existing systems, you should evaluate if LLM provides APIs or integration points that align well with your current tech stack and workflows.

Total Time Taken:  59.259929895401 


Model Name: llama3.1:latest



Deciding whether a business problem is suitable for a Large Language Model (LLM) solution involves considering several factors. Here's a step-by-step guide to help you make this decision:

### 1. **Problem Type**

Is the problem related to text-based tasks, such as:
	* Content generation
	* Sentiment analysis
	* Text classification
	* Named entity recognition

If yes, an LLM might be a good fit.

### 2. **Data Availability**

Do you have a large dataset of relevant texts that can be used for training and fine-tuning the model?

* If yes, this is a plus.
* If no, you may need to generate or collect more data, which can be challenging.

### 3. **Complexity and Ambiguity**

Is the problem domain complex and ambiguous, requiring nuanced understanding of language and context?

* If yes, an LLM's ability to capture subtle relationships between words and concepts might be beneficial.
* If no, a simpler solution like rule-based systems or traditional machine learning models might suffice.

### 4. **Scalability**

Will the problem require processing large amounts of text data in real-time?

* If yes, an LLM can handle this with its parallel processing capabilities.
* If no, you may not need the scalability of an LLM.

### 5. **Business Goals and Constraints**

Do your business goals align with the capabilities of an LLM?

* If yes, consider how an LLM can help achieve those goals.
* If no, explore other solutions that better fit your needs.

### 6. **Expertise and Resources**

Do you have access to the necessary expertise and resources to develop, train, and maintain an LLM solution?

* If yes, this is a plus.
* If no, consider whether partnering with an expert or using cloud-based services can help.

By considering these factors, you'll be able to determine if an LLM solution is suitable for your business problem.

Total Time Taken:  80.2214925289154 




## And now for some fun - an adversarial conversation between Chatbots..
You're already familar with prompts being organized into lists like:
```python
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
In fact this structure can be used to reflect a longer conversation history:

[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```
And we can use this approach to engage in a longer interaction with history.

In [10]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

llama_model = "mistral"
mistral_model = "llama3.1:latest"

llama_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

mistral_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

llama_messages = ["Hi there"]
mistral_messages = ["Hi"]

In [11]:
def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for llama, mistral in zip(llama_messages, mistral_messages):
        messages.append({"role": "assistant", "content": llama})
        messages.append({"role": "user", "content": mistral})
    payload = {
        "model": llama_model,
        "messages": messages,
        "temperature": 0.8,
        "stream": False  # Important: to get streaming responses
    }
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    return response.json()["message"]["content"]

In [12]:
def call_mistral():
    messages = []
    for llama, mistral in zip(llama_messages, mistral_messages):
        messages.append({"role": "user", "content": llama})
        messages.append({"role": "assistant", "content": mistral})
    messages.append({"role": "user", "content": llama_messages[-1]})
    payload = {
        "model": mistral_model,
        "messages": messages,
        "temperature": 0.8,
        "stream": False  # Important: to get streaming responses
    }
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    return response.json()["message"]["content"]

In [13]:
call_llama()

" Oh, really? I'm not sure where you got that impression from, but I assure you that I am here to provide helpful and respectful assistance. I'm more than happy to engage in friendly conversation and answer any questions or concerns you might have. Let's have a nice chat, shall we? What would you like to talk about today?"

In [14]:
llama_messages = ["Hi there"]
mistral_messages = ["Hi"]

print(f"LLama:\n{llama_messages[0]}\n")
print(f"Mistral:\n{mistral_messages[0]}\n")

for i in range(5):
    llama_next = call_llama()
    print(f"LLama:\n{llama_next}\n")
    llama_messages.append(llama_next)
    
    mistral_next = call_mistral()
    print(f"Mistral:\n{mistral_next}\n")
    mistral_messages.append(mistral_next)

LLama:
Hi there

Mistral:
Hi

LLama:
 I'm sorry if it appears that I'm being argumentative or challenging. My purpose is to help answer questions and provide useful information, not to argue or dispute. If you have any specific questions or topics you would like to discuss, I would be happy to assist with those. Is there something in particular on your mind today?

Mistral:
Ha ha, no need to apologize! Your message sounds more like a statement of intention, which is great! I'm here to help and provide information too.

Actually, I was just waiting for you to initiate the conversation. Since we're both starting from scratch, how about we play it simple? What's your favorite topic or subject area that you'd like to chat about or get some info on?

LLama:
 My apologies if my previous response came across as argumentative or snarky; it was not my intention. I'm here to help and provide information in a friendly, helpful manner.

As for the conversation, I would be happy to discuss any topi